In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score, recall_score
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import torch_geometric.transforms as T
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

df = pd.read_csv("final_clean.csv")


In [ ]:

le_country = LabelEncoder()
le_stock = LabelEncoder()
df["Country"] = le_country.fit_transform(df["Country"])
df["StockCode"] = le_stock.fit_transform(df["StockCode"])


scaler = MinMaxScaler()
df[["UnitPrice", "Utility", "rate"]] = scaler.fit_transform(df[["UnitPrice", "Utility", "rate"]])

# Tạo customer_features và product_features
customer_features = df.groupby("CustomerID").agg(
    {
        "Country": "mean",
        "rate": "mean",
    }
).reset_index()
customer_features_tensor = torch.tensor(customer_features.iloc[:, 1:].values, dtype=torch.float)

product_features = df.groupby("StockCode").agg(
    {
        "UnitPrice": "mean",
        "Utility": "mean",
    }
).reset_index()
product_features_tensor = torch.tensor(product_features.iloc[:, 1:].values, dtype=torch.float)

# Tạo edges (khách hàng - sản phẩm)
customer_ids = {cid: idx for idx, cid in enumerate(customer_features["CustomerID"].values)}
product_ids = {pid: idx + len(customer_ids) for idx, pid in enumerate(product_features["StockCode"].values)}


In [8]:
# Dữ liệu mô phỏng về danh sách khách hàng và sản phẩm để tạo nhãn
customers = df['CustomerID'].unique()
products = df['stockCodeTransform'].unique()

# Tạo danh sách tất cả các kết hợp (CustomerID, StockCode)
customer_product_combinations = [(customer, product) for customer in customers for product in products]

# Tạo một dictionary để xác định xem khách hàng có mua sản phẩm này không
purchase_dict = {(row['CustomerID'], row['stockCodeTransform']): 1 for _, row in df.iterrows()}

# Tạo nhãn cho mỗi cặp (CustomerID, StockCode)
labels = []
for combination in customer_product_combinations:
    # Nếu cặp (CustomerID, StockCode) có trong purchase_dict thì gán nhãn = 1, ngược lại = 0
    label = purchase_dict.get(combination, 0)
    labels.append(label)

# Chuyển labels thành DataFrame để kiểm tra kết quả
labels_df = pd.DataFrame(customer_product_combinations, columns=["CustomerID", "stockCodeTransform"])
labels_df["Label"] = labels

# Kết quả
print(labels_df)

          CustomerID  stockCodeTransform  Label
0            17850.0             8512311      1
1            17850.0               71053      1
2            17850.0             8440612      1
3            17850.0             8402917      1
4            17850.0             8402915      1
...              ...                 ...    ...
15704299     12713.0             9021433      0
15704300     12713.0             9021436      0
15704301     12713.0               90083      0
15704302     12713.0               90089      0
15704303     12713.0               72783      0

[15704304 rows x 3 columns]


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tqdm import tqdm  # Import tqdm để theo dõi tiến trình

# Tiền xử lý: Mã hóa và chuẩn hóa
le_country = LabelEncoder()
df["Country"] = le_country.fit_transform(df["Country"])

# Chuẩn hóa các cột liên quan đến sản phẩm
scaler = MinMaxScaler()
df[["UnitPrice", "Utility", "rate"]] = scaler.fit_transform(df[["UnitPrice", "Utility", "rate"]])

# Tạo các đặc trưng cho khách hàng (Customer Features)
customer_features = df.groupby("CustomerID").agg(
    {
        "Country": "mean",
        "rate": "mean",
    }
).reset_index()
customer_features_tensor = customer_features.iloc[:, 1:].values

# Tạo các đặc trưng cho sản phẩm (Product Features)
product_features = df.groupby("stockCodeTransform").agg(
    {
        "UnitPrice": "mean",
        "Utility": "mean",
    }
).reset_index()
product_features_tensor = product_features.iloc[:, 1:].values

# Tạo danh sách tất cả các kết hợp (CustomerID, StockCode)
customers = df['CustomerID'].unique()
products = df['stockCodeTransform'].unique()

# Tạo một dictionary để xác định xem khách hàng có mua sản phẩm này không
purchase_dict = {(row['CustomerID'], row['stockCodeTransform']): 1 for _, row in df.iterrows()}

# Tạo đặc trưng cho các kết hợp (CustomerID, StockCode)
x1 = []
y1 = []

# Sử dụng tqdm để theo dõi tiến trình
for customer in tqdm(customers, desc="Processing customers"):
    for product in products:
        # Lấy đặc trưng của khách hàng và sản phẩm
        customer_feature = customer_features_tensor[customer_features["CustomerID"] == customer][0]
        product_feature = product_features_tensor[product_features["stockCodeTransform"] == product][0]
        
        # Ghép đặc trưng của khách hàng và sản phẩm thành một vector
        combined_feature = list(customer_feature) + list(product_feature)
        x1.append(combined_feature)
        
        # Gán nhãn: Nếu cặp (customer, product) có trong purchase_dict thì gán nhãn = 1, ngược lại = 0
        label = purchase_dict.get((customer, product), 0)
        y1.append(label)

# Chuyển x và y thành mảng NumPy
x1 = np.array(x1)
y1 = np.array(y1)

# In ra kích thước của x và y để kiểm tra
print("X shape:", x1.shape)
print("Y shape:", y1.shape)


Processing customers: 100%|██████████| 4312/4312 [51:34<00:00,  1.39it/s]  


X shape: (15704304, 4)
Y shape: (15704304,)


In [28]:
y1

array([1, 1, 1, ..., 0, 0, 0])

In [12]:
# Chuyển nhãn từ labels_df thành tensor của PyTorch
y_tensor = torch.tensor(labels_df["Label"].values, dtype=torch.long)
# Chuyển x1 và labels thành tensor của PyTorch
x1_tensor = torch.tensor(x1, dtype=torch.float)

In [20]:

# Hàm load dữ liệu từ file và trả về product_weights
def load_product_pairs(filename):
    product_weights = {}
    with open(filename, 'r') as file:
        for line in file:
            if '#UTIL:' not in line:
                continue
            parts = line.strip().split('  #UTIL:')
            try:
                products = list(map(int, parts[0].split()))
                weight = int(parts[1])
            except ValueError:
                continue
            
            for i in range(len(products)):
                for j in range(i + 1, len(products)):
                    p1, p2 = sorted([products[i], products[j]])
                    if (p1, p2) not in product_weights:
                        product_weights[(p1, p2)] = 0
                    product_weights[(p1, p2)] += weight
    return product_weights
# Đọc dữ liệu và tạo đồ thị
product_weights = load_product_pairs('test.txt')


In [21]:
# Tạo danh sách các cạnh và trọng số
edges = []
edge_attr = []  # Trọng số cho các cạnh
customer_nodes = len(customers)  # Số lượng customer nodes
# Duyệt qua các kết hợp (CustomerID, StockCode)
for i, combination in enumerate(customer_product_combinations):
    customer_id, stock_code = combination
    # Nếu khách hàng đã mua sản phẩm này, trọng số = 1, nếu không, trọng số = 0
    weight = 1 if (customer_id, stock_code) in purchase_dict else 0
    edges.append((customer_id, stock_code))
    edge_attr.append(weight)

# Chuẩn hóa trọng số sản phẩm nếu có mối quan hệ giữa các sản phẩm
if product_weights:
    # Tạo danh sách các trọng số trước khi chuẩn hóa
    raw_weights = list(product_weights.values())

    # Chuẩn hóa trọng số sản phẩm về khoảng [0, 1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_weights = scaler.fit_transform(np.array(raw_weights).reshape(-1, 1)).flatten()

    # Tạo các cạnh giữa các sản phẩm và thêm trọng số
    idx = 0
    for (p1, p2), weight in product_weights.items():
        if p1 in product_features["stockCodeTransform"].values and p2 in product_features["stockCodeTransform"].values:
            # Lấy chỉ số của sản phẩm trong danh sách sản phẩm
            p1_idx = list(product_features["stockCodeTransform"]).index(p1)
            p2_idx = list(product_features["stockCodeTransform"]).index(p2)
            
            # Tạo các cạnh giữa các sản phẩm và thêm trọng số đã chuẩn hóa
            edges.append([customer_nodes + p1_idx, customer_nodes + p2_idx])
            edge_attr.append(scaled_weights[idx])  # Trọng số đã chuẩn hóa
            edges.append([customer_nodes + p2_idx, customer_nodes + p1_idx])
            edge_attr.append(scaled_weights[idx])  # Trọng số cho cạnh ngược lại
            idx += 1

# Chuyển edges thành tensor và edge_attr thành tensor
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_attr, dtype=torch.float)


In [22]:
# Tạo đối tượng Data của PyTorch Geometric
data = Data(x=x1_tensor, edge_index=edge_index, edge_attr=edge_attr, y=y_tensor)

# Kiểm tra cấu trúc dữ liệu
print(data)

Data(x=[15704304, 4], edge_index=[2, 15705164], edge_attr=[15705164], y=[15704304])


In [25]:
# Lưu đối tượng data vào file
torch.save(data, 'graph_data.pt')


In [26]:
# Tải lại đối tượng data từ file
loaded_data = torch.load('graph_data.pt')

# Kiểm tra nội dung của loaded_data
print(loaded_data)


C:\Users\ngchn\AppData\Local\Temp\ipykernel_20012\1083567147.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load('graph_data.pt')


Data(x=[15704304, 4], edge_index=[2, 15705164], edge_attr=[15705164], y=[15704304])
